# Candy Crush in Python - Part 6 - Putting Everything Together

In this lesson we will take all of the functions we have written so far and create our game.

# Code Recap

So far we have:
* Our game board and function to display the grid
* A function to swap two cells beside eachother
* A function to find all matches of a particular length on the board
* A function to remove matches
* A function to refill the empty spaces either with the cells from above, or random cells
* A function to handle user clicks and perform swapping, matching and refill recursively

In [1]:
import numpy as np
from gamegrid import GameGrid
from IPython.display import display
import time


def show_board(board_array):
    grid = GameGrid()
    grid.data = board_array
    display(grid)
    

def swap_items(board, first, second):    
    dist = np.sum(np.abs(np.subtract(second, first)))
    
    is_beside = dist <= 1
    
    if(is_beside):
        tmp = board[first]
        board[first] = board[second]        
        board[second] = tmp 


def horiz_match_length(board, point):
    first = board[point]
    (r, c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1
    
    for i in range(c+1, col_count):
        if(board[r, i] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count


def vert_match_length(board, point):
    first = board[point]
    (r,c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1    
    
    for i in range(r+1, row_count):
        if(board[i,c] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count


def find_matches(board, required_matches):
    horiz_matches = []
    vert_matches = []
    
    (row_count, col_count) = board.shape
    
    for r in range(row_count):
        for c in range(col_count):
            point = (r,c)
            
            horiz_len = horiz_match_length(board, point)
            if horiz_len >= required_matches:
                horiz_matches.append((r, c, horiz_len))  
            
            vert_len = vert_match_length(board, point)
            if vert_len >= required_matches:
                vert_matches.append((r, c, vert_len))
                
    return (horiz_matches, vert_matches)


def remove_matching_tiles(board, horiz_matches, vert_matches):
    for (r, c, length) in horiz_matches:
         for i in range(c, c+length):
                board[r, i] = -1
    
    for (r, c, length) in vert_matches:
        for i in range(r, r+length):
            board[i, c] = -1
            
            
def pull_down_cell(board, point, number_of_types):
    (r, c) = point
    for r_above in reversed(range(r)):    
        if(board[r_above, c] != -1):            
            board[r,c] = board[r_above,c]
            board[r_above,c] = -1
            break
    if(board[r,c] == -1):
        board[r,c] = np.random.randint(number_of_types)
        
        
def pull_down_cells(board, number_of_types):
    for r in reversed(range(board.shape[0])):
        for c in range(board.shape[1]):
            if(board[r,c] == -1):
                pull_down_cell(board, (r, c), number_of_types)
                

def update_grid(grid, board):
    grid.data = board
    time.sleep(0.25)
    
                
def process_matches(board, number_of_types, required_matches, refresh):
    
    # Pass in the required_matches
    (h_matches, v_matches) = find_matches(board, required_matches)
    remove_matching_tiles(board, h_matches, v_matches)
    match_count = len(h_matches) + len(v_matches)    
    
    if match_count > 0:        
        refresh()
        pull_down_cells(board, number_of_types)
        refresh()
        # Pass the required_matches to the next process_matches
        count = process_matches(board, number_of_types, required_matches, refresh)
        match_count = match_count + count
        
    return match_count


def handle_click(grid, point, required_matches):   
        
    grid.toggle_select(point)
    
    if len(grid.selected) < 2:
        return    
    
    board = grid.data
    refresh = lambda: update_grid(grid, board)
    last_point = grid.selected[0]
   
    swap_items(board, last_point, point)
    refresh()
    
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    number_of_types = len(grid.images)
    # Process all matches and get the total number found
    match_count = process_matches(board, number_of_types, required_matches, refresh)
    
    if(match_count == 0):
        # We havent found any matches so swap back
        swap_items(board, last_point, point)      
        refresh()

And we can use this code as follows:

In [2]:
find_all_grid = GameGrid()
find_all_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,4],
    [2,1,3,3,2],
    [4,4,1,2,3]
])

required_matches = 3

find_all_grid.on_click(
    lambda g, r, c: handle_click(g, (r, c), required_matches))

display(find_all_grid)

GameGrid()

## Create Grid Function

We are now ready to put this all together into a single function that creates our grid. So lets make a function `create_grid(...)`  that takes parameters `rows`, `cols` and `required_matches` and does the following:
- Creates a new grid of a size `rows` x `cols`
- Setup the `on_click(...)` to process matches of the size specified in the `required_matches` parameter
- Create a new board with random data
- Make sure we dont have any initial matches 
- Return the grid

In [3]:
def create_grid(rows, cols, required_matches):
        
    # Create a new grid and 
    grid = GameGrid()    
    grid.on_click(
        lambda g, r, c: handle_click(g, (r, c), required_matches))
    
    # Create a new Random Board
    number_of_types = len(grid.images)
    board = np.random.randint(number_of_types, size=(rows, cols))
    process_matches(board, number_of_types, required_matches, lambda: None)
    grid.data = board
        
    return grid

To create our random board, we use `np.random.randint(...)`. After this we must call `process_matches(...)` because we want to make sure our random grid doesnt already have any matches.

When we call `process_matches(...)`, we pass in `lambda: None` as the last parameter. Recall that the last parameter of `process_matches(...)` is a function that updates the grid on screen. We dont want to update the grid on screen when we create and process the board, so we can pass `lambda: None` as a way of saying do nothing.

We can now use our function to create a new game grid of whatever size we like, that matches whatever number of tiles we want:

In [5]:
grid = create_grid(6, 8, 4)
display(grid)

GameGrid()

And thats it! If we want to create a new game we can use call `create_grid(...)` with the number of rows, columns and required matches.

## Summary

In this lesson we learned how to put all of our components together to create the game.

This is the final tutorial, but there are a few extra notebooks that show the code for doing some other tasks:
* [Creating a New Game Button](Extras1-NewGame.ipynb)
* [Adding a Score](Extras2-Score.ipynb)
* [Using Different Images](Extras3-DifferentImages.ipynb)
* [Handling Game Over](Extras4-GameOver.ipynb)